<h1><span style = "color: blue; font-weight:bold" > Scrapping Covid Data from Ministry of Health</span></h1>

This notebook is to collect the covid situation which is happenning in Vietnam

<h2> 1. Import library</h2>

In [34]:
import pandas as pd
import numpy as np
import requests
import requests
from bs4 import BeautifulSoup
import lxml.html
import datetime
import time
import pandas as pd
import re
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import matplotlib.pyplot as plt
import seaborn as sns

<h2> 2. Scrapped Data </h2>

In [71]:
url = "https://ncov.moh.gov.vn/vi/web/guest/trang-chu?p_p_id=corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_delta=100&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_resetCur=false&_corona_trangchu_top_CoronaTrangchuTopPortlet_INSTANCE_RrVAbIFIPL7v_cur={}"

In [209]:
def get_soup_data(link, m):
    final_soup = []
    for i in tqdm(range (0,m)):
        page = requests.get(link.format(i), verify = False)
        soup = BeautifulSoup(page.text, 'html.parser')
        soup = soup.find_all('div', attrs= {"class": "inner"}) 
        final_soup.append(soup)
    return final_soup

In [210]:
soup = get_soup_data(url,100)


100%|██████████| 100/100 [04:11<00:00,  1.91s/it]


<h2> 3. Extract & Clean data </h2>

In [237]:
def clean_tag(text):
    patient_n_age = [m.find_all('a',attrs = {"class":"text-uppercase"}) for i in soup for m in i]
    patient_n_age = [i.get_text() for m in patient_n_age for i in m ]
    loc_sta_nat  = [m.find_all('p') for i in soup for m in i]
    loc_sta_nat = [i.get_text() for m in loc_sta_nat for i in m ]
    return patient_n_age, loc_sta_nat


In [238]:
patient_n_age, loc_sta_nat = clean_tag(soup)

In [239]:
patient_number = [ i.split("-")[0].replace("\t","") for i in patient_n_age ]
age = [ i.split("-")[1].replace("tuổi","") for i in patient_n_age ]
province = [ i.split("\n")[1].replace(" I","") for i in loc_sta_nat]
status = [ i.split("\n")[2].replace("I","") for i in loc_sta_nat]
nationality = [ i.split("\n")[3].replace("\r","") for i in loc_sta_nat]

In [240]:
data =pd.DataFrame(list(zip(patient_number,age,province,status,nationality)), 
                  columns = ['patient_number','age','province','status','nationality'])
    

In [241]:

data.tail()

,patient_number,age,province,status,nationality
9995,BN22,60,Quảng Ninh,Khỏi,Vương quốc Anh
9996,BN21,61,Hà Nội,Khỏi,Việt Nam
9997,BN20,28,Hà Nội,Khỏi,Việt Nam
9998,BN19,64,Hà Nội,Khỏi,Việt Nam
9999,BN18,27,Ninh Bình,Khỏi,Việt Nam


In [242]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
patient_number    10000 non-null object
age               10000 non-null object
province          10000 non-null object
status            10000 non-null object
nationality       10000 non-null object
dtypes: object(5)
memory usage: 390.8+ KB


In [243]:
data['status']

0       Đang điều trị
1       Đang điều trị
2       Đang điều trị
3       Đang điều trị
4       Đang điều trị
            ...      
9995             Khỏi
9996             Khỏi
9997             Khỏi
9998             Khỏi
9999             Khỏi
Name: status, Length: 10000, dtype: object

<h2> 4. Translate to English </h2>

In [244]:
import google_trans_new
from google_trans_new import google_translator  

Demo translate

In [245]:
translator = google_translator()  
print(translator.translate('Đang điều trị',lang_tgt='en'))

Being treated 


Loop to translate all rows in Vietnamese to English

In [246]:
translator = google_translator()
translations = {}
for column in data.columns[2:5]:
    # Unique elements of the column
    unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element,lang_tgt='en')


In [247]:
import itertools
dict(itertools.islice(translations.items(), 3)) 

{'Hồ Chí Minh': 'Ho Chi Minh ',
 'Bắc Giang': 'Bac Giang ',
 'Bắc Ninh': 'Bac Ninh '}

In [248]:
adj_dic = {'Thái Bình': 'Thai Binh','Hà Nam': 'Ha Nam ','Hoà Bình': 'Hoa Binh ',
           'Gia Lai': 'Gia Lai','Bình Định': 'Binh Dinh','Khỏi': 'Recovered',
           'Áo': 'Austria','Đức': 'Germany', 'U-dơ-bê-ki-xtan': 'Uzbekistan',
           'Ba Lan': 'Poland','Nga': 'Russia','Bra-xin': 'Brazil','Li-bi': 'Libya',
           'Nê-pan': 'Nepal ','Hung-ga-ri': 'Hungary ','Pháp': 'France ','Mi-an-ma': 'Myanmar',
          'Lát-vi-a': 'Latvia', 'Moldova': 'Moldova', 'And Islands ':'Vietnam'}

In [249]:
for i in translations.keys():
    if i in adj_dic.keys():
        translations[i] = adj_dic[i]

In [250]:
data.replace(translations, inplace = True)

In [251]:
data.head()

,patient_number,age,province,status,nationality
0,BN9917,40,Ho Chi Minh,Being treated,Vietnam
1,BN9916,28,Ho Chi Minh,Being treated,Vietnam
2,BN9915,46,Ho Chi Minh,Being treated,Vietnam
3,BN9914,45,Ho Chi Minh,Being treated,Vietnam
4,BN9913,55,Ho Chi Minh,Being treated,Vietnam


<h2> 5. Basic EDA </h2>

This is to explore answers for these questions:
1. Number of cases by province and country
2. Number of cases by status


<b>1. Number of cases by province and country</b>

In [252]:
data.groupby('province').patient_number.count().sort_values(ascending = False)

province
Bac Giang           3873
Bac Ninh            1275
Ho Chi Minh          925
Hai Duong            812
Hanoi                728
Danang               654
Khanh Hoa            143
BA Ria Vung Tau      138
Quang Nam            126
Vinh Phuc            100
Lang Son              97
Quang Ninh            90
hung Yen              75
Thai Binh             63
Kien Giang            63
Tay Ninh              62
Dien Bien             59
Ha Nam                55
Bac Lieu              53
Binh Duong            53
Ha Tinh               43
Ninh Binh             40
Can Tho               34
Hoa Binh              33
Dong Nai              31
Dong Thap             29
Phu Yen               28
Gia Lai               28
Thanh Hoa             27
Long An               26
Nam Dinh              25
Ben tre               20
An Giang              20
Soc Trang             19
Vinh Long             15
Tien Giang            14
Quang Tri             13
Ninh Thuan            12
Hai Phong             12
Binh Thuan      

In [253]:
data.groupby('nationality').patient_number.count().sort_values(ascending = False)

nationality
Vietnam                         9767
India                             60
Russia                            27
UK                                22
China                             12
USA                               11
Philippines                       11
South Africa                       8
Brazil                             7
France                             7
Germany                            6
Japan                              6
Taiwan                             5
Austria                            4
Korea                              4
Costa Rica Republic                3
Belarus                            3
Turkey                             3
Ukraine                            2
Pakistan                           2
And Islands                        2
Myanmar                            2
Israel                             2
Egypt                              1
Denmark                            1
Czech Republic                     1
Serbia and Montenegro     

<b> 2. Number of cases by status </b>

In [254]:
data.groupby('status').patient_number.count()

status
Being treated     6250
Dead                55
Other                4
Recovered         3691
Name: patient_number, dtype: int64

In [255]:
data[data.status == 'Other ']

,patient_number,age,province,status,nationality
7788,BN2229,54,Hanoi,Other,Japan
9253,BN764,67,Danang,Other,Vietnam
9564,BN453,56,Danang,Other,Vietnam
9599,BN418,61,Danang,Other,Vietnam


Status "Other" are used to refer to 4 patients that die because of other reason not covid
+ Patient 418: died because of chronic kidney disease ( 4 times negative test covid)
+ Patient 453: died because of chronic kidney disease, pneumonia, heart failure, stroke ( 3 times negative test covid)
+ Patient 764: died because of chronic kidney disease,hypertension, stroke ( 3 times negative test covid)
+ Patient 2229: died unknown reason ( 2 times negative test covid)

<b> 3. Total cases </b>

In [257]:
data

,patient_number,age,province,status,nationality
0,BN9917,40,Ho Chi Minh,Being treated,Vietnam
1,BN9916,28,Ho Chi Minh,Being treated,Vietnam
2,BN9915,46,Ho Chi Minh,Being treated,Vietnam
3,BN9914,45,Ho Chi Minh,Being treated,Vietnam
4,BN9913,55,Ho Chi Minh,Being treated,Vietnam
...,...,...,...,...,...
9995,BN22,60,Quang Ninh,Recovered,UK
9996,BN21,61,Hanoi,Recovered,Vietnam
9997,BN20,28,Hanoi,Recovered,Vietnam
9998,BN19,64,Hanoi,Recovered,Vietnam
